<a href="https://colab.research.google.com/github/hana-an/Image-quality-metrics/blob/main/docs/examples/model_energy_U_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Energy estimation of U-Net model.

Unet shows reduction in total energy per inference (GPU) and reduction in storage requirements.



In [ ]:
!pip install keras_spiking

In [3]:
import warnings

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import TensorBoard

import keras_spiking

warnings.simplefilter("ignore")
tf.get_logger().addFilter(lambda rec: "Tracing is expensive" not in rec.msg)

## Using ModelEnergy

The ``keras_spiking.ModelEnergy`` class provides the entry point for energy estimation.
It takes a Keras model as input, and computes relevant statistics for that model.

unet

In [4]:

num_classes = 1

inputs = tf.keras.layers.Input((128,128,3))

#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
b1 = tf.keras.layers.BatchNormalization()(c1)
r1 = tf.keras.layers.ReLU()(b1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(r1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
b2 = tf.keras.layers.BatchNormalization()(c2)
r2 = tf.keras.layers.ReLU()(b2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(r2)

c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
b3 = tf.keras.layers.BatchNormalization()(c3)
r3 = tf.keras.layers.ReLU()(b3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(r3)

c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
b4 = tf.keras.layers.BatchNormalization()(c4)
r4 = tf.keras.layers.ReLU()(b4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(r4)

c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
b5 = tf.keras.layers.BatchNormalization()(c5)
r5 = tf.keras.layers.ReLU()(b5)
c5 = tf.keras.layers.Dropout(0.3)(r5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Expansive path
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
u6 = tf.keras.layers.BatchNormalization()(u6)
u6 = tf.keras.layers.ReLU()(u6)


u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(u6)
u7 = tf.keras.layers.concatenate([u7, c3])
u7 = tf.keras.layers.BatchNormalization()(u7)
u7 = tf.keras.layers.ReLU()(u7)


u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(u7)
u8 = tf.keras.layers.concatenate([u8, c2])
u8 = tf.keras.layers.BatchNormalization()(u8)
u8 = tf.keras.layers.ReLU()(u8)

u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(u8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
u9 = tf.keras.layers.BatchNormalization()(u9)
u9 = tf.keras.layers.ReLU()(u9)


outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='sigmoid')(u9)
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 16)         448       ['input_1[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 128, 128, 16)         0         ['conv2d[0][0]']              
                                                                                                  
 conv2d_1 (Conv2D)           (None, 128, 128, 16)         2320      ['dropout[0][0]']             
                                                                                              

In [5]:
# estimate model energy
energy = keras_spiking.ModelEnergy(model)
energy.summary(print_warnings=False)

Layer (type)                           |Output shape        |Param #|Conn #  |Neuron #|J/inf (cpu)
---------------------------------------|--------------------|-------|--------|--------|-----------
input_1 (InputLayer)                   |[(None, 128, 128, 3)|      0|       0|       0|          0
conv2d (Conv2D)                        |(None, 128, 128, 16)|    448| 7077888|  262144|      0.063
dropout (Dropout)                      |(None, 128, 128, 16)|      0|       0|       0|          0
conv2d_1 (Conv2D)                      |(None, 128, 128, 16)|   2320|37748736|  262144|       0.33
batch_normalization (BatchNormalization|(None, 128, 128, 16)|     64|       0|       0|          0
re_lu (ReLU)                           |(None, 128, 128, 16)|      0|       0|  262144|     0.0023
max_pooling2d (MaxPooling2D)           |  (None, 64, 64, 16)|      0|       0|       0|          0
conv2d_2 (Conv2D)                      |  (None, 64, 64, 32)|   4640|18874368|  131072|       0.16
dropout_1 

In [6]:
energy.summary(
    columns=(
        "name",
        "energy cpu",
        "energy gpu",
        "synop_energy cpu",
        "synop_energy gpu",
        "neuron_energy cpu",
        "neuron_energy gpu",
    ),
    print_warnings=False,
)

Layer (type)      |J/inf (cpu)|J/inf (gpu)|Synop J/inf (|Synop J/inf (|Neuron J/inf |Neuron J/inf 
------------------|-----------|-----------|-------------|-------------|-------------|-------------
input_1 (InputLaye|          0|          0|            0|            0|            0|            0
conv2d (Conv2D)   |      0.063|     0.0022|        0.061|       0.0021|       0.0023|      7.9e-05
dropout (Dropout) |          0|          0|            0|            0|            0|            0
conv2d_1 (Conv2D) |       0.33|      0.011|         0.32|        0.011|       0.0023|      7.9e-05
batch_normalizatio|          0|          0|            0|            0|            0|            0
re_lu (ReLU)      |     0.0023|    7.9e-05|            0|            0|       0.0023|      7.9e-05
max_pooling2d (Max|          0|          0|            0|            0|            0|            0
conv2d_2 (Conv2D) |       0.16|     0.0057|         0.16|       0.0057|       0.0011|      3.9e-05
dropout_1 

lcrm unet

In [7]:


num_classes = 1

inputs = tf.keras.layers.Input((128,128,3))

#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=3,strides=(1,1),padding='same')(c1)
#c1= #tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
b1 = tf.keras.layers.BatchNormalization()(c1)
r1 = tf.keras.layers.ReLU()(b1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(r1)

c2 = tf.keras.layers.DepthwiseConv2D(depth_multiplier=2,kernel_size=3,strides=(1,1),padding='same')(p1)
#c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=3,strides=(1,1),padding='same')(c2)
#c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
b2 = tf.keras.layers.BatchNormalization()(c2)
r2 = tf.keras.layers.ReLU()(b2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(r2)

c3 = tf.keras.layers.DepthwiseConv2D(depth_multiplier=2,kernel_size=3,strides=(1,1),padding='same')(p2)
#c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)

c3 = tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=3,strides=(1,1),padding='same')(c3)
#c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
b3 = tf.keras.layers.BatchNormalization()(c3)
r3 = tf.keras.layers.ReLU()(b3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(r3)

c3 = tf.keras.layers.DepthwiseConv2D(depth_multiplier=2,kernel_size=3,strides=(1,1),padding='same')(p2)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)

c4 = tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=3,strides=(1,1),padding='same')(c4)
#c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
b4 = tf.keras.layers.BatchNormalization()(c4)
r4 = tf.keras.layers.ReLU()(b4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(r4)

c5 = tf.keras.layers.DepthwiseConv2D(depth_multiplier=2,kernel_size=3,strides=(1,1),padding='same')(p4)
#c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
b5 = tf.keras.layers.BatchNormalization()(c5)
r5 = tf.keras.layers.ReLU()(b5)
c5 = tf.keras.layers.Dropout(0.3)(r5)
c5 = tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=3,strides=(1,1),padding='same')(c5)
#c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Expansive path
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
u6 = tf.keras.layers.BatchNormalization()(u6)
u6 = tf.keras.layers.ReLU()(u6)


u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(u6)
u7 = tf.keras.layers.concatenate([u7, c3])
u7 = tf.keras.layers.BatchNormalization()(u7)
u7 = tf.keras.layers.ReLU()(u7)


u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(u7)
u8 = tf.keras.layers.concatenate([u8, c2])
u8 = tf.keras.layers.BatchNormalization()(u8)
u8 = tf.keras.layers.ReLU()(u8)

u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(u8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
u9 = tf.keras.layers.BatchNormalization()(u9)
u9 = tf.keras.layers.ReLU()(u9)


outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='sigmoid')(u9)
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

In [9]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d_11 (Conv2D)          (None, 128, 128, 16)         448       ['input_2[0][0]']             
                                                                                                  
 dropout_5 (Dropout)         (None, 128, 128, 16)         0         ['conv2d_11[0][0]']           
                                                                                                  
 depthwise_conv2d (Depthwis  (None, 128, 128, 16)         160       ['dropout_5[0][0]']           
 eConv2D)                                                                                   

In [10]:
# estimate model energy
energy = keras_spiking.ModelEnergy(model)
energy.summary(print_warnings=False)

Layer (type)                           |Output shape        |Param #|Conn #  |Neuron #|J/inf (cpu)
---------------------------------------|--------------------|-------|--------|--------|-----------
input_2 (InputLayer)                   |[(None, 128, 128, 3)|      0|       0|       0|          0
conv2d_11 (Conv2D)                     |(None, 128, 128, 16)|    448| 7077888|  262144|      0.063
dropout_5 (Dropout)                    |(None, 128, 128, 16)|      0|       0|       0|          0
depthwise_conv2d (DepthwiseConv2D)     |(None, 128, 128, 16)|    160|       0|       0|          0
batch_normalization_9 (BatchNormalizati|(None, 128, 128, 16)|     64|       0|       0|          0
re_lu_9 (ReLU)                         |(None, 128, 128, 16)|      0|       0|  262144|     0.0023
max_pooling2d_4 (MaxPooling2D)         |  (None, 64, 64, 16)|      0|       0|       0|          0
depthwise_conv2d_1 (DepthwiseConv2D)   |  (None, 64, 64, 32)|    320|       0|       0|          0
dropout_6 

In [11]:
energy.summary(
    columns=(
        "name",
        "energy cpu",
        "energy gpu",
        "synop_energy cpu",
        "synop_energy gpu",
        "neuron_energy cpu",
        "neuron_energy gpu",
    ),
    print_warnings=False,
)

Layer (type)      |J/inf (cpu)|J/inf (gpu)|Synop J/inf (|Synop J/inf (|Neuron J/inf |Neuron J/inf 
------------------|-----------|-----------|-------------|-------------|-------------|-------------
input_2 (InputLaye|          0|          0|            0|            0|            0|            0
conv2d_11 (Conv2D)|      0.063|     0.0022|        0.061|       0.0021|       0.0023|      7.9e-05
dropout_5 (Dropout|          0|          0|            0|            0|            0|            0
depthwise_conv2d (|          0|          0|            0|            0|            0|            0
batch_normalizatio|          0|          0|            0|            0|            0|            0
re_lu_9 (ReLU)    |     0.0023|    7.9e-05|            0|            0|       0.0023|      7.9e-05
max_pooling2d_4 (M|          0|          0|            0|            0|            0|            0
depthwise_conv2d_1|          0|          0|            0|            0|            0|            0
dropout_6 

# **Result Summary of LCRM (optimized) AlexNEt:**
Total params: 7787658 (29.71 MB)

Trainable params: 7784970 (29.70 MB)

Non-trainable params: 2688 (10.50 KB)

Total energy per inference [Joules/inf] (cpu): 8.15e-01

Total energy per inference [Joules/inf] (gpu): 2.84e-02

# **Result Summary of parent ALexNet model:**
Total params: 14250762 (54.36 MB)

Trainable params: 14248010 (54.35 MB)

Non-trainable params: 2752 (10.75 KB)

Total energy per inference [Joules/inf] (cpu): 9.35e+00

Total energy per inference [Joules/inf] (gpu): 3.26e-01



**Overall there is a 91.2 % reduction in total energy perinference (GPU) and 45.3 % reduction in storage requirement**